In [71]:
import pandas as pd
import numpy as np
import gensim
import datetime as dt
import re
import string
import nltk
from nltk import word_tokenize, tokenize

In [2]:
biden = pd.read_csv('../data/hashtag_joebiden.csv')
trump = pd.read_csv('../data/hashtag_donaldtrump.csv', lineterminator='\n')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
full = biden.append(trump)

In [35]:
# deleting weird rows
full = full.dropna()

# clean up datetime objects
full['created_at'] = pd.to_datetime(full['created_at'])
full.shape

(345224, 21)

#### Include only tweets from before election day

In [40]:
full = full[full['created_at'] <= dt.datetime(2020, 11, 3)]
full.shape

(161470, 21)

#### Looking at the texts

In [46]:
all_tweets = full.tweet.to_list()
all_tweets

['In 2020, #NYPost is being #censorship #CENSORED by Twitter to manipulate a US election in favor of #JoeBiden and against #Trump.\n\nbut CCP from #China or porn on Twitter? \n\nThat’s always been fine for @jack @vijaya @dickc @KatieS.\n\n@marciadorsey is @jack sick?',
 'Comments on this? "Do Democrats Understand how Ruthless China is?" https://t.co/QevK00yhs3 #China #HunterBiden #JoeBiden #BidenHarris #BidenHarris2020 #TrumpPence2020 #Trump #realDonaldTrump #WTO #coronavirus #trade https://t.co/Wjm1hyTFNZ',
 '@RealJamesWoods #BidenCrimeFamily #JoeBiden #HunterBiden #HunterBidenEmails https://t.co/ottX1yP37j',
 '#Trump #Obama #Clinton #Biden\n\n#ManWomanPersonCameraTV\n\n#RockPaperScissors\n\n#HandsFaceSpace\n\n#VoteJoeBiden \n\n#USElection #USElection2020\n\n#Pence vs #Harris\n\n#DrugsTest #TrumpDrugs #BidenDrugs\n\n#InjectDisinfectant #PowerfulLight through the skin or some other way 😳🙆🏽\u200d♂️🤷🏽\u200d♂️🤦🏽\u200d♂️ https://t.co/tBqSpgWoR0',
 'Come on @ABC PLEASE DO THE RIGHT THING. M

#### Strip away all URLs

In [58]:
all_tweets = [re.sub(r'http\S+', '', t) for t in all_tweets]

#### Strip away next lines and other string literals

In [59]:
all_tweets = [re.sub(r'\n', '', t) for t in all_tweets]
all_tweets

['In 2020, #NYPost is being #censorship #CENSORED by Twitter to manipulate a US election in favor of #JoeBiden and against #Trump.but CCP from #China or porn on Twitter? That’s always been fine for @jack @vijaya @dickc @KatieS.@marciadorsey is @jack sick?',
 'Comments on this? "Do Democrats Understand how Ruthless China is?"  #China #HunterBiden #JoeBiden #BidenHarris #BidenHarris2020 #TrumpPence2020 #Trump #realDonaldTrump #WTO #coronavirus #trade ',
 '@RealJamesWoods #BidenCrimeFamily #JoeBiden #HunterBiden #HunterBidenEmails ',
 '#Trump #Obama #Clinton #Biden#ManWomanPersonCameraTV#RockPaperScissors#HandsFaceSpace#VoteJoeBiden #USElection #USElection2020#Pence vs #Harris#DrugsTest #TrumpDrugs #BidenDrugs#InjectDisinfectant #PowerfulLight through the skin or some other way 😳🙆🏽\u200d♂️🤷🏽\u200d♂️🤦🏽\u200d♂️ ',
 'Come on @ABC PLEASE DO THE RIGHT THING. Move the #biden town hall.  I bet a requirement for Orange Monkey to do the NBC town hall was to have it opposite Biden.  PLEASE MOVE YOU

In [62]:
nopunct = []
for sent in all_tweets:
    a = [w for w in sent.split() if w not in string.punctuation]
    nopunct.append(" ".join(a))
nopunct

['In 2020, #NYPost is being #censorship #CENSORED by Twitter to manipulate a US election in favor of #JoeBiden and against #Trump.but CCP from #China or porn on Twitter? That’s always been fine for @jack @vijaya @dickc @KatieS.@marciadorsey is @jack sick?',
 'Comments on this? "Do Democrats Understand how Ruthless China is?" #China #HunterBiden #JoeBiden #BidenHarris #BidenHarris2020 #TrumpPence2020 #Trump #realDonaldTrump #WTO #coronavirus #trade',
 '@RealJamesWoods #BidenCrimeFamily #JoeBiden #HunterBiden #HunterBidenEmails',
 '#Trump #Obama #Clinton #Biden#ManWomanPersonCameraTV#RockPaperScissors#HandsFaceSpace#VoteJoeBiden #USElection #USElection2020#Pence vs #Harris#DrugsTest #TrumpDrugs #BidenDrugs#InjectDisinfectant #PowerfulLight through the skin or some other way 😳🙆🏽\u200d♂️🤷🏽\u200d♂️🤦🏽\u200d♂️',
 'Come on @ABC PLEASE DO THE RIGHT THING. Move the #biden town hall. I bet a requirement for Orange Monkey to do the NBC town hall was to have it opposite Biden. PLEASE MOVE YOURS.',


In [72]:
tok_corp = [nltk.word_tokenize(sent) for sent in nopunct]

In [73]:
unique_words = list(set([item for sublist in tok_corp for item in sublist]))
len(unique_words)

213974

In [74]:
model = gensim.models.Word2Vec(tok_corp, min_count=1, size=16, window=5)

In [75]:
vector_list = model[unique_words]

<ipython-input-75-3b9ece09146b>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector_list = model[unique_words]


In [92]:
model.wv.most_similar(positive=['trump', 'biden'], negative=['pence'])

[('returnsWe', 0.8606514930725098),
 ('reign.', 0.8408111929893494),
 ('Paul_lander', 0.8358066082000732),
 ('cut.Now', 0.7910617589950562),
 ('ride.', 0.7874192595481873),
 ('cornmaze', 0.78668212890625),
 ('abogamos', 0.7819547057151794),
 ('.ALL', 0.7798870205879211),
 ('ConspiracyTheory.Next', 0.7792178392410278),
 ('Stocking', 0.77637779712677)]

In [94]:
model.wv.most_similar(positive=['trump', 'biden'], negative=['election'])

[('Crossed', 0.8647177219390869),
 ('joebiden', 0.8573160767555237),
 ('SomewherAbove', 0.8514740467071533),
 ('🤷😂😂😂', 0.8313897848129272),
 ('donaldtrump', 0.8277565836906433),
 ('bidenfracking', 0.8273472785949707),
 ('apart.Do', 0.7946151494979858),
 ('twitter', 0.7926451563835144),
 ('blackownedbusiness', 0.7904813289642334),
 ('recesión', 0.790270209312439)]

In [95]:
model.wv.most_similar(positive=['donaldtrump', 'joebiden'], negative=['election'])

[('BlackVotes', 0.8624715209007263),
 ('asymmetric', 0.8429577946662903),
 ('MahachieJosey', 0.8236658573150635),
 ('WaterDean', 0.8187867999076843),
 ('Goodbyeriddance', 0.8149021863937378),
 ('quackery++TRULY', 0.8041679263114929),
 ('alexperruzzi', 0.8034356236457825),
 ('recesión', 0.8029419183731079),
 ('latam', 0.8026280403137207),
 ('itsince', 0.800908625125885)]

In [96]:
model.wv.most_similar(positive=['donaldtrump', 'joebiden'], negative=['win'])

[('BlackVotes', 0.8455004692077637),
 ('Goodbyeriddance', 0.8431543111801147),
 ('asymmetric', 0.8152742385864258),
 ('WaterDean', 0.8148641586303711),
 ('BermasBrigade', 0.8041631579399109),
 ('joebidenforpresident2020', 0.7998307943344116),
 ('ordonnance', 0.79508376121521),
 ('terrorisminfrance', 0.7938036918640137),
 ('AbsolutBex', 0.7922503352165222),
 ('apart.Do', 0.7906863689422607)]

In [97]:
model.wv.most_similar(positive=['joe', 'kamala'], negative=['president'])

[('விமர்சித்துள்ளார்.முழுமையாக', 0.91624915599823),
 ('zzxyxy', 0.9069697260856628),
 ('Godtv', 0.8992291688919067),
 ('bidenlies', 0.8923174738883972),
 ('mikeyharlow', 0.8884468078613281),
 ('salsadancing', 0.8882232904434204),
 ('AntifaBLM', 0.8862686157226562),
 ('FrankDLucas', 0.8862311244010925),
 ('lilwayne', 0.885867178440094),
 ('debate2020', 0.8854920864105225)]

In [98]:
model.wv.most_similar(positive=['democrats', 'republicans'], negative=['joebiden'])

[('parachute', 0.8096869587898254),
 ('RightWing', 0.8071244955062866),
 ('realdonaldtrump.Trump', 0.8059526681900024),
 ('leisure.', 0.8039405345916748),
 ('Biden2000', 0.8022629618644714),
 ('trumpcard', 0.7957147359848022),
 ('troupe', 0.7938852310180664),
 ('TRUMPKIN', 0.7914862036705017),
 ('republican', 0.7903239727020264),
 ('senate', 0.7807750105857849)]

In [99]:
model.wv.most_similar(positive=['republicans', 'democrats'], negative=['donaldtrump'])

[('democracy', 0.8234735131263733),
 ('leisure.', 0.8138745427131653),
 ('parachute', 0.8105553984642029),
 ('TRUMPKIN', 0.8006261587142944),
 ('BothWaysBiden', 0.7815749645233154),
 ('gop', 0.779120922088623),
 ('RightWing', 0.7776183485984802),
 ('ImpeachTrump', 0.7772533297538757),
 ('mentalhealthawareness', 0.7735905051231384),
 ('Republicans', 0.7727471590042114)]

In [100]:
model.wv.most_similar(positive=['republicans', 'democrats'], negative=['fake'])

[('OOPSY', 0.8672438263893127),
 ('MalkamDior', 0.8552485704421997),
 ('castyourvote', 0.8491604924201965),
 ('castyourballot', 0.8450116515159607),
 ('MitchMcZombie', 0.8446080684661865),
 ('copwatchpatrolunit', 0.8412543535232544),
 ('democrat', 0.8348299264907837),
 ('bethevoice', 0.8216090202331543),
 ('vote2018', 0.8205949664115906),
 ('trumpcard', 0.8126664161682129)]

In [101]:
model.wv.most_similar(positive=['republicans', 'democrats'], negative=['steal'])

[('love♥️', 0.850869357585907),
 ('forabolso', 0.8419191837310791),
 ('raven_salve', 0.8377131223678589),
 ('joeb…', 0.8194279670715332),
 ('betcanada', 0.8108905553817749),
 ('Chris_Meloni', 0.7924087047576904),
 ('AndrewDahDude', 0.7909923791885376),
 ('corporatists', 0.7885407209396362),
 ('he+', 0.7882601618766785),
 ('AbsolutBex', 0.7829505801200867)]

In [102]:
model.wv.most_similar(positive=['trump', 'biden'], negative=['steal'])

[('joebiden', 0.9055683612823486),
 ('donaldtrump', 0.8370770215988159),
 ('TopicTuesday', 0.8012052178382874),
 ('SomewherAbove', 0.7888171076774597),
 ('m_millsey', 0.7826275825500488),
 ('BermasBrigade', 0.7724413275718689),
 ('UX', 0.7674868702888489),
 ('apart.Do', 0.762323260307312),
 ('joebidenforpresident2020', 0.7423136234283447),
 ('medios.', 0.7387043833732605)]

In [103]:
model.wv.most_similar(positive=['trump', 'biden'], negative=['fake'])

[('bidenfracking', 0.8508836030960083),
 ('WORSEPRESIDENTEVER', 0.8435609936714172),
 ('joebidenforpresident2020', 0.8425644636154175),
 ('settleforbiden', 0.8420864343643188),
 ('blackownedbusiness', 0.8377708196640015),
 ('digitalforensics', 0.8328734636306763),
 ('ConsejoDeSeguridad', 0.8314307928085327),
 ('thewire', 0.8313229084014893),
 ('blue🌊', 0.8206116557121277),
 ('JoeBidenforpresident2020🙏🏽', 0.8107916116714478)]

In [104]:
model.wv.most_similar(positive=['trump', 'biden'], negative=['red'])

[('bidenfracking', 0.8744525909423828),
 ('joebiden', 0.8717766404151917),
 ('SomewherAbove', 0.8637628555297852),
 ('TrumpLatestNews', 0.8550676107406616),
 ('comedy_entertainment', 0.8549621105194092),
 ('donaldtrump', 0.8205596208572388),
 ('letsmakeamericagreatagain', 0.8159626126289368),
 ('BERNIE2020', 0.8151552677154541),
 ('douyin', 0.8103376626968384),
 ('DonaldtrumpOK', 0.8012486696243286)]

In [108]:
model.wv.most_similar(positive=['trump', 'biden'], negative=['fraud'])

[('WORSEPRESIDENTEVER', 0.822706937789917),
 ('Ensuring', 0.8013590574264526),
 ('joebiden', 0.7996094822883606),
 ('DemocratsAreMarxistsDestroyingAmerica', 0.7981131076812744),
 ('nowHopefully', 0.796665608882904),
 ('🎃🇺🇸', 0.7956656217575073),
 ('fall.we', 0.7925233840942383),
 ('bidenfracking', 0.7877020239830017),
 ('wearescrewed', 0.771823525428772),
 ('m_millsey', 0.7687368392944336)]

In [111]:
model.wv.most_similar(positive=['fuckdonaldtrump', 'biden'], negative=['trump'])

[('ridinwithbiden', 0.9681268334388733),
 ('BidenforMD', 0.9617548584938049),
 ('blackandwhite', 0.9571888446807861),
 ('TonyBubolinski', 0.9545313715934753),
 ('uselection2020', 0.9517863392829895),
 ('wealthplanning', 0.9500529766082764),
 ('familyoffice', 0.9491546154022217),
 ('Demonstrators', 0.9478327035903931),
 ('IndiaUSPartnership', 0.9473299384117126),
 ('mikelove', 0.9470313787460327)]

In [112]:
model.wv.most_similar(positive=['trump', 'biden'], negative=['black'])

[('bidenfracking', 0.898440420627594),
 ('SomewherAbove', 0.8716250658035278),
 ('digitalforensics', 0.8556708693504333),
 ('joebiden', 0.847095787525177),
 ('settleforbiden', 0.8371075391769409),
 ('BERNIE2020', 0.8297250866889954),
 ('joebidenforpresident2020', 0.8276945352554321),
 ('letsmakeamericagreatagain', 0.827506959438324),
 ('🎃🇺🇸', 0.8271589875221252),
 ('comedy_entertainment', 0.824272632598877)]

In [113]:
model.wv.most_similar(positive=['trump', 'biden'], negative=['MAGA'])

[('joebiden', 0.8784961700439453),
 ('donaldtrump', 0.8736578226089478),
 ('SomewherAbove', 0.8317824006080627),
 ('joebidenforpresident2020', 0.8014957904815674),
 ('-even', 0.7861276865005493),
 ('afianzarse', 0.7855063676834106),
 ('BermasBrigade', 0.7799546718597412),
 ('AmericaEvenGreater', 0.7723953127861023),
 ('220M', 0.7642503976821899),
 ('163-year', 0.7639360427856445)]

In [116]:
model.wv.most_similar(positive=['biden', 'trump'], negative=['bidenfracking'])

[('better', 0.8414340019226074),
 ('this', 0.8283975124359131),
 ('tRump', 0.8256000280380249),
 ('real', 0.824661135673523),
 ('Aint', 0.8217821717262268),
 ('yet', 0.8201666474342346),
 ('election', 0.8187140226364136),
 ('it', 0.8153673410415649),
 ('either', 0.8151133060455322),
 ('again', 0.8130486011505127)]